# Computervisie

In deze week gaan we vooral focussen op een aantal technieken om te werken met visuele inputs.
Als eerste gaan we een aantal technieken zien hoe je deze data op een eenvoudige manier kan inlezen en verwerken.
Daarna gaan we een aantal veelgebruikte technieken zijn om meer data aan te maken via varianten van bestaande data.
Ten slotte ga je leren hoe je op een efficiente manier modellen kan trainen om classificatie of regressie uit te voeren op een dataset van beelden.

## Data inlezen

Vorige week heb je geleerd om gestructureerde data in te lezen uit csv-bestanden.
Om datasets voor computervisie in te lezen is het vaak complexer omdat typisch elke input in een aparte file staat (1 per beeldje).
Het label/target van de figuur kan dan in een csv-bestand staan, in de naam van het bestand of folder staan.
Hoe deze data ingeladen wordt hangt dus af van de dataset.
In deze notebook staan een aantal voorbeelden om dit te doen met pytorch en met keras.

### Pytorch

Hieronder staan de meest voorkomende voorbeelden om dit te doen met pytorch:
* Veel gebruikte klassieke datasets vind je in de torchvision package, meer bepaald onder datasets.
* Via de ImageFolder klasse in het geval dat elke klasse in een aparte subfolder aanwezig is
* Je maakt een eigen custom-dataset klasse dat bepaald hoe de figuren ingeladen worden en wat hun label is.

Hieronder staan code-voorbeelden van deze technieken

In [ ]:
import torch
from torchvision import datasets, transforms

# Transforms om beelden voor te bereiden
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

# Load the dataset
trainset = datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=4, shuffle=True, num_workers=2)

In [ ]:
transform = transforms.Compose([
    transforms.Resize((128, 128)),  # Verander de grootte van de afbeelding
    transforms.ToTensor(),  # Zet de afbeelding om naar een tensor
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))  # Normaliseer de afbeelding
])

# Laad de dataset
dataset = datasets.ImageFolder(root='dataset', transform=transform)

# Creëer een DataLoader om batches van gegevens te laden
dataloader = torch.utils.data.DataLoader(dataset, batch_size=32, shuffle=True)

# Voorbeeld van het verkrijgen van een batch van gegevens
for images, labels in dataloader:
    print(images.size(), labels.size())
    break

In [ ]:
from PIL import Image
import os
from torch.utils.data import Dataset, DataLoader

class CustomImageDataset(Dataset):
    def __init__(self, image_dir, transform=None):
        self.root_dir = image_dir
        self.transform = transform
        
        # Verkrijg de lijst van subfolders (klassen)
        self.classes = sorted(os.listdir(image_dir))
        self.class_to_idx = {cls: idx for idx, cls in enumerate(self.classes)}
        
        # Verkrijg de lijst van alle afbeeldingen en hun labels
        self.image_paths = []
        self.labels = []
        for cls in self.classes:
            cls_dir = os.path.join(image_dir, cls)
            for img_name in os.listdir(cls_dir):
                self.image_paths.append(os.path.join(cls_dir, img_name))
                self.labels.append(self.class_to_idx[cls])
    
    def __len__(self):
        return len(self.image_paths)
    
    def __getitem__(self, idx):
        img_path = self.image_paths[idx]
        image = Image.open(img_path).convert('RGB')
        label = self.labels[idx]
        
        # Pas transformaties toe als die zijn gedefinieerd
        if self.transform:
            image = self.transform(image)
        
        return image, label

# Definieer transformaties
transform = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

# Creëer een dataset en DataLoader
dataset = CustomImageDataset(image_dir='dataset', transform=transform)
dataloader = DataLoader(dataset, batch_size=32, shuffle=True)

# Voorbeeld van het verkrijgen van een batch van gegevens
for images, labels in dataloader:
    print(images.size(), labels.size())
    break

### Keras

De technieken in Keras om dit uit te voeren zijn gelijkaardig.

Hieronder staan de meest voorkomende voorbeelden om dit te doen met pytorch:
* Veel gebruikte klassieke datasets vind je in de torchvision package, meer bepaald onder datasets.
* Via de ImageFolder klasse in het geval dat elke klasse in een aparte subfolder aanwezig is
* Je maakt een eigen custom-dataset klasse dat bepaald hoe de figuren ingeladen worden en wat hun label is.

Hieronder staan code-voorbeelden van deze technieken

In [ ]:
import tensorflow as tf
from keras.datasets import cifar10

# Laad de CIFAR-10 dataset
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

# Normaliseer de afbeeldingen
x_train, x_test = x_train / 255.0, x_test / 255.0

# Maak datasets voor training en validatie
train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train))
test_dataset = tf.data.Dataset.from_tensor_slices((x_test, y_test))

# Voorbeeld van het verkrijgen van een batch van gegevens
for images, labels in train_dataset.take(1):
    print(images.shape, labels.shape)

In [ ]:
import keras

# Definieer de dataset directory en batchgrootte
dataset_dir = 'dataset'
batch_size = 32
img_height, img_width = 128, 128

# Laad de dataset
train_dataset = tf.keras.preprocessing.image_dataset_from_directory(
    dataset_dir,
    validation_split=0.2,  # Verdeel in train en validatie
    subset="training",
    seed=123,
    image_size=(img_height, img_width),
    batch_size=batch_size
)

# Voorbeeld van het verkrijgen van een batch van gegevens
for images, labels in train_dataset.take(1):
    print(images.shape, labels.shape)

In [ ]:
import tensorflow as tf
import os
from PIL import Image
import numpy as np

class CustomImageDataset(tf.data.Dataset):
    def __new__(cls, image_dir, image_size=(128, 128), batch_size=32):
        # Verkrijg alle submappen en hun labels
        class_names = sorted(os.listdir(image_dir))
        class_to_label = {class_name: idx for idx, class_name in enumerate(class_names)}
        
        # Verkrijg de paden en labels voor elke afbeelding
        image_paths = []
        labels = []
        for class_name in class_names:
            class_path = os.path.join(image_dir, class_name)
            for img_name in os.listdir(class_path):
                image_paths.append(os.path.join(class_path, img_name))
                labels.append(class_to_label[class_name])
        
        # Functie om een afbeelding te laden en te normaliseren
        def load_image(image_path, label):
            image = Image.open(image_path).convert('RGB')
            image = image.resize(image_size)
            image = np.array(image) / 255.0  # Normaliseer de afbeelding
            return image, label
        
        # Generator functie om data op te halen
        def generator():
            for img_path, lbl in zip(image_paths, labels):
                yield load_image(img_path, lbl)
        
        # Creëer een TensorFlow dataset van de generator
        dataset = tf.data.Dataset.from_generator(
            generator,
            output_signature=(
                tf.TensorSpec(shape=(image_size[0], image_size[1], 3), dtype=tf.float32),
                tf.TensorSpec(shape=(), dtype=tf.int32)
            )
        )
        
        # Batch en shuffle de data
        dataset = dataset.batch(batch_size).shuffle(buffer_size=1000)
        return dataset

# Voorbeeld van het gebruik van de CustomImageDataset
dataset_dir = 'dataset'
dataset = CustomImageDataset(image_dir=dataset_dir)

# Voorbeeld van het verkrijgen van een batch van gegevens
for images, labels in dataset.take(1):
    print(images.shape, labels.shape)
